# load csv file

In [110]:
import agate

artworks = agate.Table.from_csv('uniq_artworks.csv')

In [83]:
print(artworks)

| column                  | data_type |
| ----------------------- | --------- |
| _id                     | Number    |
| acquisition_year        | Number    |
| acquisition_mode        | Text      |
| date_creation           | Text      |
| type                    | Text      |
| domain                  | Text      |
| domain_leaf             | Text      |
| domain_deno_for_grid    | Text      |
| domain_description_mst  | Text      |
| mst                     | Text      |
| collection              | Text      |
| collection_department   | Text      |
| recap_authors           | Text      |
| inventory               | Text      |
| authors_list            | Text      |
| authors_nationality     | Text      |
| authors_birth_death     | Text      |
| authors_name_complement | Text      |
| title_notice            | Text      |
| title_list              | Text      |
| comments                | Text      |
| recap_description       | Text      |
| provenance              | Text      |


# Filter >= 1945 and sort by acquisition_year, acquisition_mode

In [111]:
artworks = artworks.where(lambda row: row['acquisition_year'] >= 1945).order_by('acquisition_year')
artworks_by_authors_list = artworks.group_by('acquisition_mode').aggregate([('occ',agate.Count())]).order_by('occ',True)

In [82]:
artworks_by_authors_list.limit(20).print_table()

| acquisition_mode     |    occ |
| -------------------- | ------ |
| Achat                | 27 368 |
| Achat par commande   |  2 550 |
| Don                  |  1 602 |
| Achat en salon       |  1 498 |
| Inscription à l'i... |  1 173 |
| Donation             |    442 |
| Achat en vente pu... |    168 |
| Mode d'acquisitio... |     30 |
| Attribution Minis... |     22 |
| Achat par préemption |     11 |
| Legs                 |     10 |
| Mode d'acquisitio... |      6 |
| Echange              |      5 |
| Dation               |      4 |
| Remplacement aprè... |      3 |
| Achat par plusieu... |      1 |
| Inscription à l'i... |      1 |


# Split artists groups

In [113]:
multiple_artists = artworks.where(lambda row: row['authors_list'] and ',' in row['authors_list']).select(['acquisition_year','authors_list','acquisition_mode'])
multiple_artists.group_by('authors_list').aggregate([('occ',agate.Count())]).order_by('occ',True).print_table()
print("multiple_artists legnth :%s"%len(multiple_artists.rows))
splited_artists_groups = []
for row in multiple_artists:
    for a in row['authors_list'].split(','):
        splited_artists_groups.append({
            'authors_list': a,
            'acquisition_mode': row['acquisition_mode'],
            'acquisition_year': row['acquisition_year']
        })
new_rows = agate.Table.from_object(splited_artists_groups)
print("new rows legnth :%s"%len(new_rows.rows))
print("artworks before length :%s"%len(artworks.rows))
artworks_single_authors = artworks.where(lambda row: row['authors_list'] and ',' not in row['authors_list']).select(['acquisition_year','authors_list','acquisition_mode'])
artworks_single_authors = agate.Table.merge([artworks_single_authors,new_rows])
print("artworks after length :%s"%len(artworks_single_authors.rows))

| authors_list         | occ |
| -------------------- | --- |
| JORN Asger (JORGE... |  90 |
| CARZOU Jean (ZOUL... |  32 |
| VILLON Jacques (D... |  30 |
| MOON Sarah (HADEN... |  30 |
| DELAUNAY Sonia (S... |  26 |
| SALGADO Sebastiao... |  25 |
| DI ROSA Hervé, DI... |  25 |
| RENÉ-JACQUES (GIT... |  24 |
| WEISS Sabine (WEB... |  23 |
| AUJAME Jean (AUJA... |  22 |
| SZAFRAN Sam (BERG... |  21 |
| VOLTI (VOLTI Anto... |  20 |
| LIPSI Morice (LIP... |  20 |
| CHAUVIN (CHAUVIN ... |  20 |
| SZÉKÉLY Véra (HAR... |  20 |
| LOTIRON Robert (L... |  19 |
| TAL-COAT (JACOB P... |  19 |
| KRASNO Rodolfo (K... |  19 |
| MODEL Lisette (SE... |  19 |
| ARP Jean (ARP Han... |  18 |
| ...                  | ... |
multiple_artists legnth :3965
new rows legnth :8238
artworks before length :34894
artworks after length :39121


# group by artists and transform acquisition in letters

In [120]:
artworks_by_authors_list = artworks.group_by('authors_list')

renameAcquisitionName = {'Achat':'A','Achat par commande':'C','Don':'D','Donation':'D'}

authors_list_acquisition_mode = artworks_by_authors_list.aggregate(
    [("acquisition_sequence",
      agate.Summary("acquisition_mode", agate.Text(),
                    lambda col : ''.join([renameAcquisitionName[am] for am in col.values_without_nulls() if am in renameAcquisitionName]),False)
     )]
).where(lambda row : row['acquisition_sequence']and row['acquisition_sequence'] != '')

authors_list_acquisition_mode.group_by('acquisition_sequence').aggregate([('occ',agate.Count())]).where(lambda row: row['acquisition_sequence'] and ('D' in row['acquisition_sequence'] or 'C' in row['acquisition_sequence']) ).order_by('occ',True).print_table()

| acquisition_sequence | occ |
| -------------------- | --- |
| C                    | 431 |
| D                    | 211 |
| AC                   |  79 |
| CA                   |  63 |
| CC                   |  51 |
| AAC                  |  43 |
| AAAC                 |  34 |
| AD                   |  33 |
| CAA                  |  32 |
| DD                   |  32 |
| ACA                  |  25 |
| ACAA                 |  21 |
| AAD                  |  21 |
| AACA                 |  21 |
| CCC                  |  20 |
| DA                   |  20 |
| DDD                  |  19 |
| AAAAC                |  14 |
| CAAAA                |  13 |
| CAAA                 |  13 |
| ...                  | ... |


In [118]:
authors_list_acquisition_mode.print_table()

| authors_list         | acquisition_sequence |
| -------------------- | -------------------- |
| COCHET Gérard        | AACAAAAAAAAAAAA      |
| CAILLETTE René-Jean  | AAAA                 |
| CAVAILLON Elisée     | AAAA                 |
| ABBAL André          | AACAC                |
| DEJEAN Louis         | CACCC                |
| TOUSSAINT Gaston     | A                    |
| DEBARRE Jean René    | AAA                  |
| RIGAL André Louis... | A                    |
| YENCESSE Ovide       | A                    |
| D'AMBROSIO Louis     | A                    |
| COULON Georges       | ACCC                 |
| GIBERT Lucien        | ACCACA               |
| RISPAL Gabriel       | CCCC                 |
| JOACHIM Jean (JOA... | AACC                 |
| LENOIR Pierre        | AC                   |
| MARTIN Raymond       | ACCCCCA              |
| GILI Marcel          | ACCAACAAAA           |
| DELUOL André         | CAACCCCACA           |
| CONDOY (GARCIA Ho... | CD             